In [3]:
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

In [4]:
csv_path = 'https://raw.githubusercontent.com/uzmabb182/Data602_Assignments/main/resources/posgres_covid_df.csv'
covid_df = pd.read_csv(csv_path, encoding="utf-8")
covid_df.head(5)

,county,fips,state,year,month,cases_count,deaths_count
0,Abbeville,45001,South Carolina,2021,1,46891,834
1,Abbeville,45001,South Carolina,2021,2,58234,918
2,Abbeville,45001,South Carolina,2021,3,72427,1105
3,Abbeville,45001,South Carolina,2021,4,75231,1162
4,Abbeville,45001,South Carolina,2021,5,79939,1271


In [9]:
csv_path = 'https://raw.githubusercontent.com/uzmabb182/Data602_Assignments/main/resources/posgres_modality_df.csv'
modality_fips_df = pd.read_csv(csv_path, encoding="utf-8")
modality_fips_df

,countyname,stcountyfp,learning_modality,operational_schools,state,zip_code,year,month,avg_student_count
0,Abbeville,45001,Hybrid,9,SC,29620,2022,1,2965
1,Abbeville,45001,Hybrid,14,SC,29649,2022,1,8925
2,Abbeville,45001,In Person,3,SC,29692,2021,8,951
3,Abbeville,45001,In Person,3,SC,29692,2021,9,951
4,Abbeville,45001,In Person,3,SC,29692,2021,10,951
...,...,...,...,...,...,...,...,...,...
269034,Zavala,48507,In Person,5,TX,78839,2022,9,1910
269035,Zavala,48507,In Person,5,TX,78839,2022,10,1910
269036,Ziebach,46137,In Person,3,SD,57623,2022,8,396
269037,Ziebach,46137,In Person,3,SD,57623,2022,9,396
